Traditional multilayer perceptrons reboot their understanding of langauges every time it's called upon, unlike a human who has memory

A Recurrent Neural Networks (RNNs) implement a similar mechanism, making neural networks Turing complete

#### Applications
* Speech recognition
* Machine translation
* Music composition
* Handwriting recognition
* Grammar learning

For some classes of data, the order in which the observations are recieved is important:
1. "I'm sorry... it's not you, it's me"
2. "It's not me, it's you... I'm sorry"

In a normal feed forward network, the hidden layers provide a useful intermediate representation, which each layer transforming the input data into another representation to make it easier to solve the problem. The **hidden state** is the representation being stored in the hidden layers.

When considering the next time-step in the sequence, we want to leverage any information we've already extracted from the sequence, by calculating the next hidden state as a combination of the previous hidden state and latest input

One method of combining is as follows
1. The representation of the prevous item in the sequence is calculated as $w_{ih}x^{(t=1)}_{hidden}$
2. The representation of the next item in the sequence is calculated as $w_{ih}x^{(t=2)}_{hidden}$
3. The representation of the prevous time step is combined as $x^{(t)}_{hidden}=tanh(w_{hh}x^{(t-1)}_{hidden}+w_{ih}x^{(t)}_{input})$
4. Repeat as long as necessary for an arbirarily long sequence of observations

Notice the different set of weights used for the previous state and the current state

Other methods of combining include:
* Gated recurrent units
* Long short-term memory units

By remembering the previous hidden state, we can backpropagate to the earlier timesteps, "backpropagation through time." If training a vanilla neural network is optimization over functions, training recurrent nets is optimization over programs

# Common Structures
The ability to handle an arbitrary length input and outputs allows the RNN to tackle a broad range of tasks
## One to many
Generating an arbitrary length sequence from a fixed length input
* Image captioning
The prediction from each time step is fed in as input to the enxt timestep

## Many to one
* Sentiment from a text
Generally used to do classification on a sequence of data

## Many to many (same)
Predicting a lable for each observation in a sequence; **dense classification**
* Detect named entiies (person, organization, location) in sentences
* Predicting the current activity in a frame of a video

## Many to many (different)
Translates a sequence of inputs to a different but related sequence of outputs

Both the inputs and the outputs are abritrary length sequence

* Machine translation

## Bidirectionality
A weakness in an ordinary RNN is being able to use only the observations previously seen. For example, a model for recognizing named entity recognition:

* "I can't believe that Teddy Roosevelt was your great grandfather!"
* "I can't believe that Teddy bear is made out of chocolate!"

In this case, it's very difficult to tell if Teddy is the president or the bear if you're only going from the front to back. The model needs to be able to see what comes after the current token, to determine if it's the start of a name

A **bidirectional** recurrent network processes the sequence left-to-right and right-to-left in parallel then combines these two representation such that at any point in the sequence, the model has knowledge of the tokens which can before AND after it

# Limitations
The layer's weights are SHARED across time-steps, so when a RNN makes a mistake, the error signal needs to back many time-steps

Because the signal is being multiplied by the same weight many time over, if it's something like 0.5, after 10 time steps, the error signal has changed by a factor of $0.5^{10} = 0.00098$, drastically reducing the magnitude of the error signal: **vanishing gradient problem** A large weight can have a similar problem, but can be managed through gradient clipping.

This makes training with an RNN rather slow

# Computation
Accepts an input vector x, and outputs vector y, but influenced by the entire history of inputs

A forward pass of a RNN layer might look like this

* Calculating the new history based on the previous history

$a^{(t)} = b + Wh^{(t-1)} + Ux^{(t)}$

$h^{(t)} = tanh(a^{(t)})$

* Calculating the output from the layer

$ o^{(t)} = c + Vh(t) $

* Calculating the classification from that

$ \hat{y}^{(t)} = softmax(o^{(t)}) $

### Dataset
Very small dataset of sentiment classification

In [5]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}


# Create the vocabulary
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size) # 18 unique words found

# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }

print(word_to_idx)
print(idx_to_word)

18 unique words found
{'earlier': 0, 'or': 1, 'not': 2, 'sad': 3, 'all': 4, 'at': 5, 'happy': 6, 'very': 7, 'now': 8, 'was': 9, 'this': 10, 'is': 11, 'and': 12, 'right': 13, 'i': 14, 'good': 15, 'bad': 16, 'am': 17}
{0: 'earlier', 1: 'or', 2: 'not', 3: 'sad', 4: 'all', 5: 'at', 6: 'happy', 7: 'very', 8: 'now', 9: 'was', 10: 'this', 11: 'is', 12: 'and', 13: 'right', 14: 'i', 15: 'good', 16: 'bad', 17: 'am'}


In [15]:
import numpy as np

def createInputs(text):
    # turns the text input into a one hot array
    inputs = []
    
    for word in text.split(' '):
        working = np.zeros((vocab_size, 1))
        working[word_to_idx[word]] = 1
        inputs.append(working)
        
    return inputs

In [1]:
class Layer_Recurrent:
    # ...
    def forward(self, inputs):
        # remember input values
        self.inputs = inputs
        
        # update the hidden state
        self.h = np.tanh(np.dot(self.history, self.weights_hh) +
                         np.dot(inputs, self.weights_xh) +
                         self.bias_h)
        
        # output vector
        self.output = np.dot(self.history, self.weights_hy) + self.bias_y

The tanh function is user here because it squishes the output between \[-1; 1\]

But multiple RNN layers can be layered one after another

In [ ]:
recurrent_1.forward(X)
recurrent_2.forward(recurrent_1.output)

Neither layers know or care that the other is recurrent layer

In [17]:
class RNN:
    
    def __init__(self, input_size, output_size, hidden_size = 64):
        
        # weights
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.001
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.001
        self.Why = np.random.randn(output_size, hidden_size) * 0.001

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))

        # Perform each step of the RNN
        for i, x in enumerate(inputs):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)

        # Compute the output
        y = self.Why @ h + self.by

        return y, h

In action:

In [31]:
def softmax(xs):
    # Applies the Softmax Function to the input array.
    return np.exp(xs) / sum(np.exp(xs))

# initialization of the model
model = RNN(vocab_size, 2)

inputs = createInputs("i am very good")
out, h = model.forward(inputs)
probs = softmax(out)

probs

array([[0.49999525],
       [0.50000475]])

### Backward Pass
The layer needs to remember every history for the time steps, and every input for the time steps

Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time step

#### Loss
But first, a very quick implementation of categorical cross-entropy loss

#### Remembering


In [29]:
class RNN:
    
    def __init__(self, input_size, output_size, hidden_size = 64):
        
        # weights
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.001
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.001
        self.Why = np.random.randn(output_size, hidden_size) * 0.001

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        
        self.last_inputs = inputs
        self.last_hs = {0: h}

        # Perform each step of the RNN
        for i, x in enumerate(inputs):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h

        # Compute the output
        y = self.Why @ h + self.by

        return y, h
    
    def backward(self, dvalues, learn_rate = 2e-2):
        pass


#### Gradient
Since the derivative of the gradient, $\frac{\partial L}{\partial y_i}$, comes out as 

* $p_i$ when $i \neq c$
* $p_i - 1$ when $i = c$

Where $p$ is the probability of a given index

In [33]:
# looping over each training example
for x, y in train_data.items():
    inputs = createInputs(x)
    target = int(y)
    
    # forward
    out, _ = model.forward(inputs)
    probs = softmax(out)
    
    # build dL/dy
    dL_dy = probs
    dL_dy[target] -= 1
    
    # backward
    model.backward(dL_dy)

In [34]:
class RNN:
    
    def __init__(self, input_size, output_size, hidden_size = 64):
        
        # weights
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.001
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.001
        self.Why = np.random.randn(output_size, hidden_size) * 0.001

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        
        self.last_inputs = inputs
        self.last_hs = {0: h}

        # Perform each step of the RNN
        for i, x in enumerate(inputs):
            h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h

        # Compute the output
        y = self.Why @ h + self.by

        return y, h
    
    def backward(self, dvalues, learn_rate = 2e-2):
        
        n = len(self.last_inputs)
        
        d_Why = dvalues @ self.last_hs[n].T
        d_by = dvalues
        
        # Because Wxh affects every h_t, which all affect y
        # we need to backpropagate through all timesteps
        # and take the sum of all of the gradients we get
        
        # initializing each of the needed gradients as zero
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh  = np.zeros(self.bh.shape)
        
        # calculate dL/dh for the last h
        dh = self.Why.T @ dvalues
        
        # backpropagate through time
        for t in reversed(range(n)):
            # intermediate value
            # dL/dsum = dL/dh * dh/dsum
            # dvalue * (1 - h^2)
            temp = dh * ((1 - self.last_hs[t + 1] ** 2))
            
            # dL/dbh = dL/h * dh/dsum * dsum/dbh
            d_bh += temp
            
            # dL/dWhh = dL/h * dh/dsum * dsum/dWhh
            d_Whh += temp @ self.last_hs[t].T
            
            # dL/dWxh = dL/h * dh/dsum * dsum/x
            d_Wxh += temp @ self.last_inputs[t].T
            
            # next dL/dh = dL/dh * dh/dsum * dsum/dh
            d_h = self.Whh @ temp
            
        # clip to prevent exploding gradients
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
            np.clip(d, -1, 1, out = d)
        
        # update the weights and biases using gradient descent
        self.Whh -= learn_rate * d_Whh
        self.Wxh -= learn_rate * d_Wxh
        self.Why -= learn_rate * d_Why
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by

In [35]:
import random

model = RNN(vocab_size, 2)

def processData(data, backprop = True):
    items = list(data.items())
    random.shuffle(items)
    
    loss = 0
    num_correct = 0
    
    for x, y in items:
        inputs = createInputs(x)
        target = int(y)
        
        # forward
        out, _ = model.forward(inputs)
        probs = softmax(out)
        
        # calculate loss/accuracy
        loss -= np.log(probs[target])
        num_correct += int(np.argmax(probs) == target)
        
        if backprop:
            dL_dy = probs
            dL_dy[target] -= 1
        
            model.backward(dL_dy)
        
    return loss / len(data), num_correct / len(data)

### Training Loop

In [40]:
for epoch in range(1000):
    train_loss, train_acc = processData(train_data)
    
    if epoch % 100 == 99:
        '''
        print(f'====Epoch: {epoch}, ' +
              f'loss: {train_loss:.3f}, ' +
              f'accuracy: {train_acc:.3f}')
        '''
        print('--- Epoch %d' % (epoch + 1))
        print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

        test_loss, test_acc = processData(test_data, backprop = False)
        print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.589 | Accuracy: 0.672
Test:	Loss 0.609 | Accuracy: 0.800
--- Epoch 200
Train:	Loss 0.423 | Accuracy: 0.845
Test:	Loss 0.480 | Accuracy: 0.800
--- Epoch 300
Train:	Loss 0.268 | Accuracy: 0.914
Test:	Loss 0.259 | Accuracy: 0.950
--- Epoch 400
Train:	Loss 0.013 | Accuracy: 1.000
Test:	Loss 0.014 | Accuracy: 1.000
--- Epoch 500
Train:	Loss 0.005 | Accuracy: 1.000
Test:	Loss 0.005 | Accuracy: 1.000
--- Epoch 600
Train:	Loss 0.357 | Accuracy: 0.862
Test:	Loss 0.706 | Accuracy: 0.550
--- Epoch 700
Train:	Loss 0.009 | Accuracy: 1.000
Test:	Loss 0.033 | Accuracy: 1.000
--- Epoch 800
Train:	Loss 0.005 | Accuracy: 1.000
Test:	Loss 0.034 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.003 | Accuracy: 1.000
Test:	Loss 0.043 | Accuracy: 0.950
--- Epoch 1000
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.054 | Accuracy: 0.950


# Long Short-Term Memory (LSTM) Network
A better performing variant of the RNN, that calculates its history differently, and has appealing backpropagation dynamics, but uses a lot more computational resources

Solves the problem with standard RNNs, which fail to learn when there are 5 to 10 discrete time steps of lag in the input

If a sequence is long enough, the model will have difficulties carrying information from earlier time steps to later ones

Solves the problems of vanishing gradients and exploding gradients

The hidden layers are fully self connected, contains memory cells and corresponding gate units

An LSTM layer consists of a set of recurrently connected blocks; **memory blocks**, that can be thought of as differentiable versions of memory chips of a computer. Each one contains one or more recurrently connected memory cells, and three multiplicative units:

* the input
* the output
* the forget gates

These units provide continuous analogues of write, read, and rest operations for the cells

The cells can only be interacted with via the gates

The gates can learn which data in a sequence is important and to keep or to throw away, so it passes relevant information down the long chain of sequences to make predictions

Learning rate and network size are the most crucial tunable hyperparameters, meaning they can be tuned independently. So the learning rate can be calibrated first using a small network, saving lot of experimentation time

## Eg
When trying to decipher the sentiment from the following review:

"Amazing! This box of cereal gave me a perfectly balances breakfast, as all things should be. I only ate half of it, but will definitely be buying again."

A human, after reading the review, will prolly remember the main points and key words such as

* Amazing
* perfectly balanced breakfast
* will definitely be buying again

And forgetting a lot of words you don't care about like "this", "gave", "all", etc etc

This is what a LSTM does

## Applications
Can be used for any sequential task, in which a hierarchical decomposition may exist, but unknown to us

Effective at capturing long-term temporal dependencies without suffering from the optimization problems of simple recurrent networks

State of the art for many difficult problems

* language modeling
* speech recognition
* acoustic modeling of speech
* speech synthesis
* machine translation
* handwriting recognition and generation
* protein secondary structure prediction
* audio analysis

## Bidirectional LSTMs
Present the training sequence fowards and backwards to two separate recurrent nets, both of which are connected to the same output layer

So the BRNN has a complete knowledge of all the points before and after it. Also nulls the need to find a time-window or target delay size because the net is free to use as much or as little of this context as necessary

Even problems such as speech recognition, for which a BRNN seems to violate causality, extremely helpful because humans make sense of sounds and words in light of the context revealed in the future

The output of a current timestep is generated from the combination of both layers' hidden vectors

## seq2seq LSTMs || RNN Encoder-Decoders
One LSTM reads the input sequence, one timestep at a time, creating a large fixed-dimensional vector representation

Another LSTM extracts the output sequence from that vector

The LSTM's ability to successfully learn from data with long range temporal dependencies makes it an excellent choice for this application, because of the large time lag between the inputs and their corresponding outputs

The encoder can be replaced by a CNN, first trained for image classification task, then the last hidden layer feeding into an RNN decoder that generates sentences

## Inner Workings
Tries to "remember" all the past knowledge that the network has seen so far and to "forget" irrelevant data. Done through the introduction of different activation function layers called "gates" with different purposes

Each LSTM recurrent unit maintains a vector called **Internal Cell State**, the information that was chosen to be retained by the previous LSTM recurrent unit

The gates are:
* Forget Gate (f): determines to what extent to forget the previous data
* Input Gate (i): determines the extent of information to be written onto the Internal Cell State
* Input Modulation Gate (g): a sub-part of the input gate, generally assumed to be inside the input gate
    * Modulates the information that the input gate writes onto the Internal State Cell by adding non-lineraity to the information, and making the information **Zero-Mean**
    * Done so to reduce the learning time
    * The zero-mean input has faster convergence
* Output Gate (o): determines what output (next hidden state) to generate from the current internal cell state

Works generally similarly to a SRN, but the Internal Cell State is passed foward along with the hidden state

### Workflow
1. Take the current input, the previous hidden state, and the previous internal cell state
2. Calculate the values of the four gates by
    * For each gate:
        * Calculate the parameterized vectors for the current input and the previous hidden state by element-wise multiplication with the concerned vector with the respective weights for each gate
    * Apply the respective activation function for each gate element-wise on the parameterized vectors
        * sigmoid(f)
        * sigmoid(i)
        * tanh(g)
        * sigmoid(o)
3. Calculate the current internal cell state by first calculating the element-wise multiplication vector of the input gate and the input modulation gate, then calculate the element-wise multiplication vector of the forget gate and the previous internal cell satte and then add the two vectors
4. Calculate the current hidden state by first taking the element-wise hyperbolic tanget of the current internal cell state vector and then performing element-wise multiplication with the output gate

The backpropagation is only different from the RNN because of the different mathematics, but follows the same flow